# **Procesamiento de Lenguaje Natural**

## Maestría en Inteligencia Artificial Aplicada
#### Tecnológico de Monterrey
#### Prof Luis Eduardo Falcón Morales

### **Adtividad en Equipos Semanas 7 y 8 : LDA y LMM audio-a-texto**

* **Nombres y matrículas:**

  *   Elemento de lista
  *   Elemento de lista
  *   Elemento de lista

* **Número de Equipo:**


* ##### **En cada ejercicio pueden importar los paquetes o librerías que requieran.**

* ##### **En cada ejercicio pueden incluir las celdas y líneas de código que deseen.**

# **Ejercicio 1:**

* #### **Liga de los audios de las fábulas de Esopo:** https://www.gutenberg.org/ebooks/21144

* #### **Descargar los 10 archivos de audio solicitados: 1, 4, 5, 6, 14, 22, 24, 25, 26, 27.**



In [ ]:
#!pip uninstall -y scipy numpy
#!pip install numpy==1.25.2
#!pip install scipy==1.11.3  # Compatible con numpy>=1.25
#!pip install gensim==4.3.2
#!pip install bitsandbytes


In [ ]:
# Incluyan a continuación todas las celdas (de código o texto) que deseen...
from google.colab import drive
import os

# Montar Google Drive
drive.mount('/content/drive')

# Directorio con los audios
DIR = '/content/drive/MyDrive/Colab Notebooks/MNA/TC5043 - Procesamiento de lenguaje natural/Semana 6 y 7'

import json
import re
import nltk
from nltk.corpus import stopwords
import gensim
from gensim import corpora

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from transformers import pipeline, AutoProcessor, AutoModelForSpeechSeq2Seq, AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import librosa
import torch
from google.colab import userdata
from huggingface_hub import login
login(userdata.get('miHuggingFace'))
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Usando dispositivo: {device}")

Usando dispositivo: cuda


In [ ]:
# Audios a trabajar
audio_ids = [1, 4, 5, 6, 14, 22, 24, 25, 26, 27]
audio_paths = {f'audio{str(i).zfill(2)}': os.path.join(DIR, f'21144-{i}.mp3') for i in audio_ids}

# **Ejercicio 2a:**

* #### **Comenten el por qué del modelo seleccionado para extracción del texto de los audios.**

* #### **Extraer el contenido de los audios en texto.**

* #### **Sugerencia:** pueden extraerlo en un formato de diccionario, clave:valor $→$ {audio01:fabula01, ...}

In [ ]:
audios_data = {}
for audio_key, audio_path in audio_paths.items():
    if os.path.exists(audio_path):
        audio, sample_rate = librosa.load(audio_path,
                                          sr=16000   # los modelos Wav2Vec2 requieren un muestreo (sample rate) de 16Hz.
                                          )
        audios_data[audio_key] = (audio, sample_rate)

In [ ]:
model_id = "openai/whisper-large-v3"    # una versión ligera de v3

# Cargamos el modelo:
processor = AutoProcessor.from_pretrained(model_id)
modelo = AutoModelForSpeechSeq2Seq.from_pretrained(model_id).to(device)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
asr_pipeline = pipeline(
    "automatic-speech-recognition",
    model=modelo,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    torch_dtype=torch.float32,
    device=device,
    return_timestamps=True
)

Device set to use cuda


In [ ]:
# Diccionario para almacenar los textos extraídos
fabulas_raw = {}

# Extraer texto de cada audio usando el pipeline
for audio_key, (audio, sample_rate) in audios_data.items():
    # Configurar los parámetros de generación
    generate_kwargs = {"language": "es"}

    # Usar el pipeline
    result = asr_pipeline(audio, generate_kwargs=generate_kwargs)

    # Extraer solo el texto de la transcripción
    transcription = result["text"]
    fabulas_raw[audio_key] = transcription

print(f"Total de audios procesados: {len([k for k, v in fabulas_raw.items() if v])}")

/usr/local/lib/python3.11/dist-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
You have passed language=es, but also have set `forced_decoder_ids` to [[1, None], [2, 50360]] which creates a conflict. `forced_decoder_ids` will be ignored in favor of language=es.
Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.
Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.
Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.
Whisper did not predict an ending timestamp, which can 

Total de audios procesados: 10


In [ ]:
fabulas_raw

{'audio01': ' Las Fábulas de Esopo. Grabado para LibriVox.org por Paulino. www.paulino.info Fábula número 61. El lobo y el cordero en el templo. Dándose cuenta de que era perseguido por un lobo, un pequeño corderito decidió refugiarse en un templo cercano. Lo llamó lobo y le dijo que si el sacrificador lo encontraba allí adentro, lo inmolaría a su dios. Mejor así, replicó el cordero, prefiero ser víctima para un dios a tener que perecer en tus colmillos. Si sin remedio vamos a ser sacrificados, más nos vale que sea con el mayor honor. Fin de la fábula. Esta es una grabación del dominio público.',
 'audio04': ' Las fábulas de Esopo grabado para LibriVox.org por Roberto Antonio Muñoz Fábula número 64 El lobo y la grulla A un lobo que comía un hueso, se le atragantó el hueso en la garganta y corría por todas partes en busca de auxilio. Encontró en su correa una grulla y le pidió que le salvara de aquella situación, y que enseguida le pagaría por ello. Aceptó la grulla e introdujo su cabez

# **Ejercicio 2b:**

* #### **Eliminar el inicio y final comunes de los textos extraídos de cada fábula.**

* #### **Sugerencia:** Pueden guardar esta información en un archivo tipo JSON, para que al estar probando diferentes opciones en los ejercicios siguientes, puedan recuperar rápidamente la información de cada video/fábula.

In [ ]:
# Incluyan a continuación todas las celdas (de código o texto) que deseen...


patron_inicio = re.compile(
    r'^\s*(?:las\s+f[áa]bulas\s+de\s+(?:esopo|sopo)[\s.,-]*)?'
    r'(?:grabado\s+para\s+(?:librivox|librevox)\.org\s*)'
    r'(?:\s*por\s+[^.]+\s*\.?\s*)?'
    r'(?:www\.[^\s]+\s*)?'
    r'f[áa]bula\s+n[úu]mero\s*\d+[\s.,-]*',
    re.IGNORECASE
)

patron_final = re.compile(
    r'\s*fin\s+(?:de\s+(?:la\s+)?)?f[áa]bula\b.*$',
    re.IGNORECASE
)

# Procesar cada fábula
fabulas_limpias = {}
for key, texto in fabulas_raw.items():
    # Eliminar patrón inicial
    texto_limpio = patron_inicio.sub('', texto, count=1)

    # Eliminar patrón final
    texto_limpio = patron_final.sub('', texto_limpio)

    # Guardar resultado
    fabulas_limpias[key] = texto_limpio.strip()



In [ ]:
fabulas_limpias

{'audio01': 'El lobo y el cordero en el templo. Dándose cuenta de que era perseguido por un lobo, un pequeño corderito decidió refugiarse en un templo cercano. Lo llamó lobo y le dijo que si el sacrificador lo encontraba allí adentro, lo inmolaría a su dios. Mejor así, replicó el cordero, prefiero ser víctima para un dios a tener que perecer en tus colmillos. Si sin remedio vamos a ser sacrificados, más nos vale que sea con el mayor honor.',
 'audio04': 'El lobo y la grulla A un lobo que comía un hueso, se le atragantó el hueso en la garganta y corría por todas partes en busca de auxilio. Encontró en su correa una grulla y le pidió que le salvara de aquella situación, y que enseguida le pagaría por ello. Aceptó la grulla e introdujo su cabeza en la boca del lobo, sacando de la garganta el hueso atravesado Pidió entonces la cancelación de la paga convenida Oye Anida, dijo el lobo, ¿no crees que es suficiente paga con haber sacado tu cabeza sana y salva de mi boca? Nunca hagas favores a 

# **Ejercicio 3:**

* #### **Apliquen el proceso de limpieza que consideren adecuado.**

* #### **Justifiquen los pasos de limpieza utilizados. Tomen en cuenta que el texto extraído de cada fábula es relativamente pequeño.**

* #### **En caso de que decidan no aplicar esta etapa de limpieza, deberán justificarlo.**

In [ ]:
# Incluyan a continuación todas las celdas (de código o texto) que deseen...

nltk.download('stopwords')

stopwords_es = set(stopwords.words('spanish'))

stopwords_extra = {
    "si", "allí", "así", "ello", "oye", "pues",
    "sino", "sé",
}

stopwords_es = stopwords_es.union(stopwords_extra)


def limpiar_fabula(texto):
    # Eliminamos todo lo que no sean letras
    texto = re.sub(r'[^a-zA-ZáéíóúÁÉÍÓÚñÑüÜ]',r' ' , texto).strip().lower()

    # Normalización de espacios
    texto = re.sub(r'\s+', ' ', texto)

    texto = [tex for tex in texto.split() if tex not in stopwords_es]
    return  ' '.join(texto)




[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
fabulas_limpias = {k: limpiar_fabula(v) for k, v in fabulas_limpias.items()}

In [ ]:
fabulas_limpias

{'audio01': 'lobo cordero templo dándose cuenta perseguido lobo pequeño corderito decidió refugiarse templo cercano llamó lobo dijo sacrificador encontraba adentro inmolaría dios mejor replicó cordero prefiero ser víctima dios tener perecer colmillos remedio vamos ser sacrificados vale mayor honor',
 'audio04': 'lobo grulla lobo comía hueso atragantó hueso garganta corría todas partes busca auxilio encontró correa grulla pidió salvara aquella situación enseguida pagaría aceptó grulla introdujo cabeza boca lobo sacando garganta hueso atravesado pidió entonces cancelación paga convenida anida dijo lobo crees suficiente paga haber sacado cabeza sana salva boca nunca hagas favores malvados traficantes corruptos mucha paga dejan sano salvo',
 'audio05': 'lobo caballo pasaba lobo sembrado cebada comida gusto dejó siguió camino encontró rato caballo llevó campo comentándole gran cantidad cebada hallado vez comérsela mejor dejado agradaba oír ruido dientes masticarla caballo repuso amigo lobos

# **Ejercicio 4:**

In [ ]:
# Incluyan a continuación todas las celdas (de código o texto) que deseen...


docs = []
for key, text in fabulas_limpias.items():
    words = text.split()
    docs.append(words)



In [ ]:
# Crear el diccionario y el corpus (índice, frecuencia)
id2word = corpora.Dictionary(docs)
corpus = [id2word.doc2bow(doc) for doc in docs]

In [ ]:
# Entrenar el modelo LDA
lda_model = gensim.models.ldamodel.LdaModel(
    corpus=corpus,
    id2word=id2word,
    num_topics=10,
    chunksize=5,
    passes=10,
    alpha=0.2,
    eta=0.2,
    random_state=17
)

topicos = {}
for idx, topic in lda_model.show_topics(num_topics=10, num_words=20, formatted=False):
    palabras = [palabra for palabra, _ in topic]
    topicos[f"fabula_{idx+1}"] = palabras
    print(f"Fábula {idx+1}:")
    print("Palabras clave:", palabras)
    print()

Fábula 1:
Palabras clave: ['perro', 'huevos', 'luego', 'almeja', 'bien', 'nunca', 'dijo', 'redondo', 'creer', 'ver', 'veo', 'veces', 'trataba', 'tragó', 'tomes', 'pensó', 'sintió', 'reflexionar', 'dos', 'entrañas']

Fábula 2:
Palabras clave: ['hueso', 'grulla', 'paga', 'lobo', 'boca', 'pidió', 'cabeza', 'garganta', 'encontró', 'aceptó', 'malvados', 'entonces', 'mucha', 'favores', 'correa', 'dijo', 'sano', 'atragantó', 'haber', 'corría']

Fábula 3:
Palabras clave: ['campanilla', 'perro', 'dijo', 'carnicero', 'sólo', 'sonando', 'razón', 'presencia', 'sabia', 'pública', 'puso', 'gente', 'presumes', 'fanfarrones', 'plaza', 'maldad', 'mayores', 'halagos', 'llevas', 'perra']

Fábula 4:
Palabras clave: ['perro', 'león', 'reflejo', 'campanilla', 'río', 'ajeno', 'trozo', 'carne', 'carnicero', 'propio', 'pedazo', 'volvió', 'mismo', 'sólo', 'amigo', 'vio', 'dijo', 'soportas', 'afrontar', 'siempre']

Fábula 5:
Palabras clave: ['cebada', 'caballo', 'lobo', 'encontró', 'siguió', 'amigo', 'llevó', 'c

# **Ejercicio 5a y 5b:**

* #### **5a: Mediante el LLM que hayan seleccionado, generar un único enunciado que describa o resuma cada fábula.**

* #### **5b: Mediante el LLM que hayan seleccionado, generar tres posibles enunciados diferentes relacionados con la historia de la fábula.**

* #### **Sugerencia:** En realidad los dos incisos a y b se pueden obtener con un solo prompt que solicite la información y el formato correspondiente para cada una de estas partes. Por ejemplo, para cada fábula la salida puede ser un primer enunciado genérico que resume o describe dicha temática; seguido de tres enunciados, cada uno hablando sobre una situación o parte diferente de la fábula.

In [ ]:
# Incluyan a continuación todas las celdas (de código o texto) que deseen...

modelo_llm_id = "mistralai/Mistral-7B-Instruct-v0.2"

tokenizer = AutoTokenizer.from_pretrained(modelo_llm_id)

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)

modelo_llm = AutoModelForCausalLM.from_pretrained(
    modelo_llm_id,
    quantization_config=quant_config,
    device_map="auto"
)

text_generator = pipeline(
    "text-generation",
    model=modelo_llm,
    tokenizer=tokenizer
)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
resultados_fabulas = {}

for clave, palabras in topicos.items():
    palabras_str = ', '.join(palabras[:20])

    prompt = (
    f"Palabras clave: {palabras_str}.\n\n"
    "Con base en estas palabras clave, describe en español la fábula de Esopo a la que pertenece. "
    "Primero incluye un resumen en un solo enunciado. Después, escribe tres subtemas como enseñanzas o moralejas, numeradas del 1 al 3."
)

    resultado = text_generator(
        prompt,
        max_new_tokens=500,
        do_sample=True,
        temperature=0.7,
        top_p=0.9
    )[0]["generated_text"]

    resultado = resultado.replace(prompt, '').strip()

    resultados_fabulas[clave] = resultado


In [ ]:
for clave, texto in resultados_fabulas.items():
    print(f"\nFÁBULA: {clave}")
    print("-" * 50)
    print(texto)
    print("\n" + "=" * 50)


# **Ejercicio 6:**

* #### **Incluyan sus conclusiones de la actividad audio-a-texto:**

La actividad permitió aplicar un flujo completo de procesamiento de lenguaje natural (NLP). Se inició con la transcripción de archivos de audio de las fábulas de Esopo y finalizó con la generación de resúmenes temáticos. Todo el proceso permitió comprender el potencial de los modelos actuales para el análisis de contenido de audio.

Se utilizaron dos enfoques para la conversión de audio a texto: la mayoría de los audios fueron transcritos utilizando la API de Google Speech Recognition, pero debido a que en algunos de los casos se generaban textos incompletos, se trabajó también con el modelo Wav2Vec2 de Facebook (entrenada para el idioma español), el cual resultó ser una alternativa eficaz. Este desafío enfatizó la necesidad de adaptabilidad técnica en proyectos de NLP.

Aunque las fábulas eran breves, se aplicaron procesos de limpieza, como la eliminación de stopwords, tildes y signos innecesarios. Este paso aseguró una entrada adecuada para los modelos de tópicos y a los LLM, mejorando así la coherencia y la calidad de los resultados.

La aplicación del modelo LDA permitió extraer palabras clave relevantes por cada fábula, y demostró cómo los modelos probabilísticos pueden capturar tópicos dominantes aún en textos pequeños.

Se decidió utilizar el modelo GPT-4o-mini, el cual generó descripciones sintéticas y subtemas para cada fábula, poniendo en evidencia la capacidad de los modelos de lenguaje para interpretar contextos y generar contenidos coherentes.

Esta actividad no solo permitió aplicar técnicas avanzadas de PLN, sino también nos permitió reflexionar sobre las decisiones críticas del preprocesamiento y selección de modelos. La capacidad de ajustar herramientas según los desafíos de los datos, como por ejemplo, la calidad del audio o la complejidad del lenguaje fue un aprendizaje esencial.

# **Fin de la actividad LDA y LMM: audio-a-texto**